# Template NER Application

In [ ]:
pip install -q tensorboardX transformers simpletransformers

     |████████████████████████████████| 124 kB 9.0 MB/s 
     |████████████████████████████████| 3.4 MB 58.5 MB/s 
     |████████████████████████████████| 247 kB 69.5 MB/s 
     |████████████████████████████████| 596 kB 59.4 MB/s 
     |████████████████████████████████| 895 kB 61.8 MB/s 
     |████████████████████████████████| 61 kB 462 kB/s 
     |████████████████████████████████| 3.3 MB 55.6 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 9.2 MB 38.3 MB/s 
     |████████████████████████████████| 306 kB 46.2 MB/s 
     |████████████████████████████████| 1.7 MB 36.0 MB/s 
     |████████████████████████████████| 1.2 MB 47.3 MB/s 
     |████████████████████████████████| 142 kB 60.2 MB/s 
     |████████████████████████████████| 97 kB 6.6 MB/s 
     |████████████████████████████████| 180 kB 58.1 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 243 kB 53.2 MB/s 
     |█████████████████

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
import os 
os.chdir("/content/drive/MyDrive/DataScienceLab/NER/templateNER")
os.listdir()

In [ ]:
import pandas as pd
import logging
from seq2seq_model import Seq2SeqModel
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
!ls ./data/IBMNew/

dev.csv       Train1K.csv  Train32K.csv  train50K.csv
Train16K.csv  Train2K.csv  Train4K.csv	 Train8K.csv


In [ ]:
train_data = pd.read_csv("./data/IBMNew/Train8K.csv", sep=';').values.tolist()
train_df = pd.DataFrame(train_data, columns=["input_text", "target_text"])
eval_data = pd.read_csv("./data/IBMNew/dev.csv", sep=';').values.tolist()
eval_df = pd.DataFrame(eval_data, columns=["input_text", "target_text"])

In [ ]:
train_df.head()

,input_text,target_text
0,As a user I expect Editing multiple artifacts ...,user is a User entity.
1,As a reviewer when I highlight a paragraph I s...,to is not a named entity.
2,"0.6: views: [direct SQL #14] As a user, I shou...",As is not a named entity.
3,[Rich Client] As a user I should be able to cr...,persist is a Verb entity.
4,"As a user, projects I create from a template w...",icon is a GUI entity.


In [ ]:
train_df.shape

(7999, 2)

In [ ]:
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 50,
    "train_batch_size": 32,
    "num_train_epochs": 5,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    "evaluate_during_training": True,
    "evaluate_generated_text": True,
    "evaluate_during_training_verbose": True,
    "use_multiprocessing": False,
    "max_length": 25,
    "manual_seed": 4,
    "save_steps": 11898,
    "gradient_accumulation_steps": 1,
    "output_dir": "./exp/mytemplate",
}

# Initialize model
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=model_args,
    # use_cuda=False,
)

In [ ]:
# Train the model
model.train_model(train_df, eval_data=eval_df)

In [ ]:
  # Evaluate the model
results = model.eval_model(eval_df, verbose=True)

In [ ]:
model.predict(["As a user I should be able to use the attribute type User in my queries."])

# SAVAS Performance Result with IBM Dataset

## bart-base performance

* 500 ..> 71
* 1000 ..> 80.3
* 2K  ..> 84.21
* 4K ..> 86.5
* 8K ..> 86.6
* 16K ..> 84.9
* 20K ..-> 86.5

## bart-large
* 1K: 86.5
* 2K : 87
* 4K: 87.1
* 8K: 87.1
* 16


